# Capstone: NFTs
## Part I: Gathering Cleaning & Cleaning

### 1. Import Libraries

In [7]:
import requests
import pandas as pd
import numpy as np
import json
import re
import os
import time
import urllib.request
from bs4 import BeautifulSoup
import bs4

### 2. Get CryptoPunk Stats & Traits Data

I will use Open Sea's API to get info on the CryptoPunks Collection:

In [8]:
# citation: https://docs.opensea.io/reference/retrieving-a-single-collection

# url for cryptopunks collection data
url = "https://api.opensea.io/api/v1/collection/cryptopunks" 

# requests
res = requests.get(url)

# convert the request into a list of dict objects
data = res.json()

# convert to dataframe
df = pd.DataFrame(data)
    
# change index so we have two columns: 'index', 'collection'
df.reset_index(inplace=True)

df

,index,collection
0,banner_image_url,https://lh3.googleusercontent.com/48oVuDyfe_xh...
1,chat_url,None
2,created_date,2019-04-26T22:13:09.691572
3,default_to_fiat,False
4,description,"CryptoPunks launched as a fixed set of 10,000 ..."
5,dev_buyer_fee_basis_points,0
6,dev_seller_fee_basis_points,0
7,discord_url,https://discord.gg/tQp4pSE
8,display_data,{'card_display_style': 'cover'}
9,editors,"[0xeaf54391793cc80de696d72713d7518c6190bfe0, 0..."


In [9]:
# only get primary_asset_contracts, stats, and traits column
small_df = df[(df["index"] == 'stats') | (df["index"] == 'traits')]
small_df

# get values in each row
stats_dict = small_df.iloc[0][1]
traits_dict = small_df.iloc[1][1]

**Stats**

In [10]:
stats_dict

{'one_day_volume': 592.49,
 'one_day_change': -0.5165872523742698,
 'one_day_sales': 6.0,
 'one_day_average_price': 98.74833333333333,
 'seven_day_volume': 9473.12,
 'seven_day_change': -0.03386774364622852,
 'seven_day_sales': 95.0,
 'seven_day_average_price': 99.71705263157895,
 'thirty_day_volume': 162881.561747886,
 'thirty_day_change': 1.9436132336455478,
 'thirty_day_sales': 341.0,
 'thirty_day_average_price': 477.6585388501056,
 'total_volume': 720096.930249314,
 'total_sales': 17798.0,
 'total_supply': 9999.0,
 'count': 9999.0,
 'num_owners': 3223,
 'average_price': 40.459429725211486,
 'num_reports': 1,
 'market_cap': 997070.809263158,
 'floor_price': None}

The value inside each key of stats dictionary is an integer. Let's convert this info to a dataframe.

In [11]:
# stats information to dataframe
stats_df = pd.DataFrame(stats_dict, index=[0]).T.reset_index()

# rename column names
stats_df.rename(columns={'index': 'stat', 0: 'amount'}, inplace = True)
stats_df

,stat,amount
0,one_day_volume,592.49
1,one_day_change,-0.516587
2,one_day_sales,6.0
3,one_day_average_price,98.748333
4,seven_day_volume,9473.12
5,seven_day_change,-0.033868
6,seven_day_sales,95.0
7,seven_day_average_price,99.717053
8,thirty_day_volume,162881.561748
9,thirty_day_change,1.943613


In [12]:
stats_df.to_csv('../data/stats.csv', index=False)

**Traits**

In [13]:
traits_dict

{'accessory': {'cap forward': 254,
  'classic shades': 502,
  'eye patch': 461,
  'rosy cheeks': 128,
  'mohawk dark': 429,
  'clown eyes blue': 384,
  'gold chain': 169,
  'pink with hat': 95,
  'peak spike': 303,
  '3 attributes': 98,
  'shaved head': 300,
  'tassle hat': 178,
  'horned rim glasses': 535,
  'straight hair blonde': 144,
  'crazy hair': 414,
  'handlebars': 263,
  'blonde short': 129,
  'eye mask': 293,
  'wild white hair': 136,
  'vampire hair': 147,
  'frumpy hair': 442,
  'buck teeth': 78,
  'nerd glasses': 572,
  'normal beard': 292,
  'front beard': 273,
  'shadow beard': 526,
  'cigarette': 961,
  'big shades': 535,
  '4 attributes': 26,
  'clown nose': 212,
  'beanie': 44,
  '1 attributes': 15,
  'wild hair': 447,
  'top hat': 115,
  'knitted cap': 419,
  'police cap': 203,
  'green eye shadow': 271,
  'vape': 272,
  'mohawk thin': 441,
  'do-rag': 300,
  'dark hair': 157,
  'purple eye shadow': 262,
  'red mohawk': 147,
  'regular shades': 527,
  'clown hair gr

The value inside each key of traits dictionary is another dictionary. We will need to go into each key of dictionary and then turn it into a dataframe.

In [14]:
# print keys of traits dictionary
for key in traits_dict.keys():
    print(key)

accessory
type


In [15]:
# print values of traits' keys
for key in traits_dict.keys():
    print(traits_dict[key])

{'cap forward': 254, 'classic shades': 502, 'eye patch': 461, 'rosy cheeks': 128, 'mohawk dark': 429, 'clown eyes blue': 384, 'gold chain': 169, 'pink with hat': 95, 'peak spike': 303, '3 attributes': 98, 'shaved head': 300, 'tassle hat': 178, 'horned rim glasses': 535, 'straight hair blonde': 144, 'crazy hair': 414, 'handlebars': 263, 'blonde short': 129, 'eye mask': 293, 'wild white hair': 136, 'vampire hair': 147, 'frumpy hair': 442, 'buck teeth': 78, 'nerd glasses': 572, 'normal beard': 292, 'front beard': 273, 'shadow beard': 526, 'cigarette': 961, 'big shades': 535, '4 attributes': 26, 'clown nose': 212, 'beanie': 44, '1 attributes': 15, 'wild hair': 447, 'top hat': 115, 'knitted cap': 419, 'police cap': 203, 'green eye shadow': 271, 'vape': 272, 'mohawk thin': 441, 'do-rag': 300, 'dark hair': 157, 'purple eye shadow': 262, 'red mohawk': 147, 'regular shades': 527, 'clown hair green': 148, 'frown': 261, 'fedora': 186, 'pigtails': 94, 'tiara': 55, 'big beard': 146, 'spots': 124, '

In [16]:
# convert type info to dataframe
type_df = pd.DataFrame(traits_dict['type'], index=[0]).T.reset_index()

# rename column names
type_df.rename(columns={'index': 'type', 0: 'amount'}, inplace= True)
type_df

# add rarity column by dividing the amount by the amount of cryptopunks, which is 10K
type_df['rarity'] = type_df['amount']/10_000
type_df

,type,amount,rarity
0,ape,24,0.0024
1,female,3840,0.3840
2,zombie,88,0.0088
3,alien,9,0.0009
4,male,6039,0.6039


In [17]:
type_df.to_csv('../data/type.csv', index=False)

In [18]:
# convert accessory info to dataframe
acc_df = pd.DataFrame(traits_dict['accessory'], index=[0]).T.reset_index()

# rename column names
acc_df.rename(columns={'index': 'accessory', 0: 'amount'}, inplace= True)

# add rarity column by dividing the amount by the amount of cryptopunks, which is 10K
pd.set_option('display.max_rows', 90)
acc_df['rarity'] = acc_df['amount']/10_000

# sort by ascending accessory name
acc_df.sort_values('accessory', inplace=True)

# reset and drop old index
acc_df.reset_index(inplace = True)
acc_df = acc_df.drop(columns = ['index'])
acc_df

,accessory,amount,rarity
0,0 attributes,2,0.0002
1,1 attributes,15,0.0015
2,2 attributes,106,0.0106
3,3 attributes,98,0.0098
4,3d glasses,286,0.0286
...,...,...,...
88,vr,332,0.0332
89,welding goggles,86,0.0086
90,wild blonde,144,0.0144
91,wild hair,447,0.0447


In [19]:
acc_df.to_csv('../data/accessory.csv', index=False)

### 3. Get Each CryptoPunk's Accessory Information

I will use a CryptoPunk Kaggle dataset to get info on the CryptoPunks Collection:

In [474]:
df = pd.read_json('../data/punks.jsonl', lines=True)

In [475]:
df.head()

,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories
0,Bid Withdrawn,0xe73a1d,,2021-09-04,2021-09-04,larvalabs,321.0,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
1,Bid,0xe73a1d,,2021-09-01,2021-09-01,larvalabs,321.0,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
2,Bid Withdrawn,0x2e5e22,,2021-09-01,2021-09-01,larvalabs,320.0,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
3,Bid,0x2e5e22,,2021-09-01,2021-09-01,larvalabs,320.0,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
4,Bid Withdrawn,0x2e5e22,,2021-09-01,2021-09-01,larvalabs,263.0,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"


In [22]:
df.shape

(167492, 12)

In [23]:
# how many in each
df['txn_type'].value_counts()

Offered            65467
Bid                26808
Sold               18979
Bid Withdrawn      16342
Transfer           14225
Offer Withdrawn    10372
Claimed            10000
(Wrap)              2773
(Unwrap)            2526
Name: txn_type, dtype: int64

In [24]:
# 10k unique punk ids
df['punk_id'].nunique()

10000

**Individual CryptoPunk with Accessory**

In [476]:
# getting only unique punks
df_claimed = df[df['txn_type'] == 'Claimed']
df_claimed = df_claimed[['punk_id', 'type', 'accessories']]

# accessories list
accessories = list(acc_df['accessory'])

# remove unneeded accessories
accessories.remove('0 attributes')
accessories.remove('1 attributes')
accessories.remove('2 attributes')
accessories.remove('3 attributes')
accessories.remove('4 attributes')
accessories.remove('5 attributes')

# create column for each accessory
for accessory in accessories:
    df_claimed[accessory] = 0

# add 1's if punk has that accessory
for i in range(len(df_claimed)):
    acc_list = df_claimed.iloc[i][2]
    for acc in acc_list:
        if acc.lower() in accessories:
            df_claimed[acc.lower()].iloc[i] = 1

/Users/olivialara/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [477]:
# create column for accessory total
df_claimed['accessory_total'] = 0

# get total accessories of each punk
for i in range(len(df_claimed)):
    acc_amount = len(df_claimed.iloc[i][2])
    df_claimed['accessory_total'].iloc[i] = acc_amount

In [478]:
# get type out of string and make lower case
for i in range(len(df_claimed)):
    df_claimed['type'].iloc[i] = df_claimed['type'].iloc[i][0]
    df_claimed['type'].iloc[i] = df_claimed['type'].iloc[i].lower()

In [484]:
# create skin color column
df_claimed['skin_color'] = df_claimed['type']
dict_skin_color = {'male': 'tan shade', 'female':'tan shade', 'zombie':'green', 'ape':'black and white', 'alien':'teal'}
df_claimed['skin_color'] = df_claimed['skin_color'].replace(dict_skin_color)

In [485]:
df_claimed['skin_color'].value_counts()

tan shade          9879
green                88
black and white      24
teal                  9
Name: skin_color, dtype: int64

In [488]:
df_claimed

,punk_id,type,accessories,3d glasses,bandana,beanie,big beard,big shades,black lipstick,blonde bob,...,top hat,vampire hair,vape,vr,welding goggles,wild blonde,wild hair,wild white hair,accessory_total,skin_color
40,0,female,"[Green Eye Shadow, Earring, Blonde Bob]",0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,3,tan shade
103,1,male,"[Smile, Mohawk]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,tan shade
133,2,female,[Wild Hair],0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,tan shade
150,3,male,"[Wild Hair, Nerd Glasses, Pipe]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,3,tan shade
162,4,male,"[Big Shades, Wild Hair, Earring, Goat]",0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,4,tan shade
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167363,9995,female,"[Purple Eye Shadow, Straight Hair Dark]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,tan shade
167368,9996,male,"[Cigarette, Earring, Crazy Hair, Smile]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,tan shade
167411,9997,zombie,"[Front Beard, Cap Forward]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,green
167439,9998,female,"[Wild White Hair, Black Lipstick, Clown Eyes G...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,3,tan shade


In [489]:
df_claimed.to_csv('../data/individual.csv', index=False)

### 4. Get Each CryptoPunk's Transaction Information

I will webscrap from Larva Labs to get each CryptoPunk's transactions:

In [202]:
# function used to find nth occurence of substring in string
# citation: https://stackoverflow.com/questions/1883980/find-the-nth-occurrence-of-substring-in-a-string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [203]:
#this for loop loops through each page of CryptoPunks' transactions to get info on its sales, bids, offers, transfers, and claims

all_trans_df = pd.DataFrame()
counter = 0

# loop through each page of All Punk Sales Pages
for num in range(0, 9_999):
    counter += 1
    num = str(num)
    url = 'https://www.larvalabs.com/cryptopunks/details/' + num + ""
    response = requests.get(url)
    response.text
    soup = BeautifulSoup(response.content)
    soup

    # GETTING TRANSACTIONS OF EACH PUNK
    # result set where class is equal to punk-history-row...
    div_sold = soup.find_all('tr', class_= 'punk-history-row-sold')
    div_bid = soup.find_all('tr', class_= 'punk-history-row-bid')
    div_offered = soup.find_all('tr', class_= 'punk-history-row-offer')
    div_transfer = soup.find_all('tr', class_= 'punk-history-row-transfer')
    div_claim = soup.find_all('tr', class_= 'punk-history-row-claim')
  
    # getting each transaction's text with list comprehension
    sales = [div_sold.text for div_sold in div_sold]
    bids = [div_bid.text for div_bid in div_bid]
    offers = [div_offered.text for div_offered in div_offered]
    transfers = [div_transfer.text for div_transfer in div_transfer]
    claims = [div_claim.text for div_claim in div_claim]

    # EMPTY DF FOR EACH CRYPTO PUNK
    trans_df = pd.DataFrame()

    # SALES DATA
    sales_usd_list = []
    sales_date_list = []

    for i in range(len(sales)):
        # get usd value
        usd = sales[i][sales[i].find('$')+1:sales[i].find(')')].replace(",", "")
        if 'M' in usd:
            usd = float(''.join(c for c in usd if c.isdigit() or c == ".")) * 1000
        # condition if less than 0.01
        elif '<' in usd:
            usd = 0.00001
        else:
            usd = float(usd)/1000
    sales_usd_list.append(usd)
    
    # date value
    start = find_nth(sales[i], '\n', 5)
    end = find_nth(sales[i], '\n', 6)
    date = sales[i][start+1:end].replace(",", "").replace(" ", "-")
    sales_date_list.append(date)

    # match sales with date    
    sales_df = pd.DataFrame(zip(sales_usd_list,sales_date_list))
    # add transaction type column
    sales_df['trans'] = 'Sold'
    # add crpto punk id column
    sales_df['id'] = int(num)

    # combine punk transactions info to one dataframe
    trans_df = trans_df.append(sales_df, ignore_index=True)

    # BID DATA
    bids_usd_list = []
    bids_date_list = []
    bids_w_usd_list = []
    bids_w_date_list = []

    for i in range(len(bids)):
        # withdrawn bid
        if bids[i][5:14] == "Withdrawn":
            # get usd value
            usd = bids[i][bids[i].find('$')+1:bids[i].find(')')].replace(",", "")
            if 'M' in usd:
                usd = float(''.join(c for c in usd if c.isdigit() or c == ".")) * 1000
            # condition if less than 0.01
            elif '<' in usd:
                usd = 0.00001
            else:
                usd = float(usd)/1000
            bids_w_usd_list.append(usd)    
            
            # date value
            start = find_nth(bids[i], '\n', 5)
            end = find_nth(bids[i], '\n', 6)
            date = bids[i][start+1:end].replace(",", "").replace(" ", "-")
            bids_w_date_list.append(date)
        # regular bid
        else:
            # get usd value
            usd = bids[i][bids[i].find('$')+1:bids[i].find(')')].replace(",", "")
            if 'M' in usd:
                usd = float(''.join(c for c in usd if c.isdigit() or c == ".")) * 1000
            elif '<' in usd:
                usd = 0.00001
            else:
                usd = float(usd)/1000
            bids_usd_list.append(usd)

            # date value
            start = find_nth(bids[i], '\n', 5)
            end = find_nth(bids[i], '\n', 6)
            date = bids[i][start+1:end].replace(",", "").replace(" ", "-")
            bids_date_list.append(date)

    # match bids with date    
    bids_df = pd.DataFrame(zip(bids_usd_list,bids_date_list))
    # add transaction type column
    bids_df['trans'] = 'Bid'
    # add crpto punk id column
    bids_df['id'] = int(num)

    # match withdrawn bids with date    
    bids_w_df = pd.DataFrame(zip(bids_w_usd_list,bids_w_date_list))
    # add transaction type column
    bids_w_df['trans'] = 'Bid Withdrawn'
    # add crpto punk id column
    bids_w_df['id'] = int(num)

    # combine punk transactions info to one dataframe
    trans_df = trans_df.append(bids_df, ignore_index=True)
    trans_df = trans_df.append(bids_w_df, ignore_index=True)

    # OFFERS DATA
    offers_usd_list = []
    offers_date_list = []
    offers_w_date_list = []
    for i in range(len(offers)):
        # withdrawn offer
        if offers[i][7:16] == "Withdrawn":
            # date value
            start = find_nth(offers[i], '\n', 5)
            end = find_nth(offers[i], '\n', 6)
            date = offers[i][start+1:end].replace(",", "").replace(" ", "-")
            offers_w_date_list.append(date)
            # regular offer
        else:
            # get usd value
            usd = offers[i][offers[i].find('$')+1:offers[i].find(')')].replace(",", "")
            if 'M' in usd:
                usd = float(''.join(c for c in usd if c.isdigit() or c == ".")) * 1_000
            elif 'B' in usd:
                usd = float(''.join(c for c in usd if c.isdigit() or c == ".")) * 1_000_000
            # eliminate offers that are a trillion or above, as these aren't actual offers
            elif ('T' in usd) | ('P' in usd) | ('E' in usd) | ('Z' in usd) | ('Y' in usd) : 
                pass
            # condition if less than 0.01
            elif '<' in usd:
                usd = 0.00001
            else:
                usd = float(usd)/1000
            offers_usd_list.append(usd)
      
            # date value
            start = find_nth(offers[i], '\n', 5)
            end = find_nth(offers[i], '\n', 6)
            date = offers[i][start+1:end].replace(",", "").replace(" ", "-")
            offers_date_list.append(date)

    # match offers with date    
    offers_df = pd.DataFrame(zip(offers_usd_list,offers_date_list))
    # add transaction type column
    offers_df['trans'] = 'Offered'
    # add crpto punk id column
    offers_df['id'] = int(num)

    # create withdrawn offers df   
    offers_w_df = pd.DataFrame()
    offers_w_df[1] = offers_w_date_list
    # add transaction type column
    offers_w_df['trans'] = 'Offer Withdrawn'
    # add crpto punk id column
    offers_w_df['id'] = int(num)

    # combine punk transactions info to one dataframe
    trans_df = trans_df.append(offers_df, ignore_index=True)
    trans_df = trans_df.append(offers_w_df, ignore_index=True)

    # TRANSFERS DATA
    transfers_date_list = []
    for i in range(len(transfers)):
        # date value
        start = find_nth(transfers[i], '\n', 5)
        end = find_nth(transfers[i], '\n', 6)
        date = transfers[i][start+1:end].replace(",", "").replace(" ", "-")
        transfers_date_list.append(date)

    # create transfers df    
    transfers_df = pd.DataFrame() 
    transfers_df[1] = transfers_date_list
    # add transaction type column
    transfers_df['trans'] = 'Transfer'
    # add crpto punk id column
    transfers_df['id'] = int(num)

    # combine punk transactions info to one dataframe
    trans_df = trans_df.append(transfers_df, ignore_index=True)

    # CLAIMS DATA
    claims_date_list = []
    for i in range(len(claims)):
        # date value
        start = find_nth(claims[i], '\n', 5)
        end = find_nth(claims[i], '\n', 6)
        date = claims[i][start+1:end].replace(",", "").replace(" ", "-")
        claims_date_list.append(date)

    # create claims df    
    claims_df = pd.DataFrame()
    claims_df[1] = claims_date_list
    # add transaction type column
    claims_df['trans'] = 'Claimed'
    # add crpto punk id column
    claims_df['id'] = int(num)

    # combine punk transactions info to one dataframe
    trans_df = trans_df.append(claims_df, ignore_index=True)

    # combine all transactions info to one dataframe
    all_trans_df = all_trans_df.append(trans_df, ignore_index=True)

# rename columns
all_trans_df.rename(columns = {0:'usd', 1: 'date'}, inplace=True)



In [158]:
# want to avoid re-running cell so will immediately save csv file
# df.to_csv('../data/transactions.csv', index=False)

In [438]:
# run this cell to see data without having to re-run the cell two cells above
df = pd.read_csv('../data/transactions.csv', index_col=0)

In [439]:
df.head()

,usd,date,trans,id
0,401.264,Aug-30-2021,Sold,9001
1,383.662,Aug-30-2021,Sold,9001
2,19.23,Feb-17-2021,Sold,9001
3,2.085,Oct-03-2020,Sold,9001
4,0.085,May-06-2019,Sold,9001
